In [1]:
from urllib import request
import json
import pandas as pd
from itertools import product
import numpy as np
from time import sleep
import time
import geopandas as gpd

## American Community Survey -- data queries
original script can be found in the repository of 'Universal patterns of long-distance commuting and social assortativity in cities' as '_script09-C_census_data_top25.ipynb'

In [2]:
# prep data to query on the top 50 metro areas
metrodata = pd.read_csv('../data/cbsa-est2018-alldata.csv', encoding = "ISO-8859-1")
metrodata = metrodata.drop_duplicates(subset=["CBSA"])
metrodata = metrodata.loc[:,["CBSA","NAME","CENSUS2010POP"]].drop_duplicates().sort_values(by="CENSUS2010POP",ascending=False).reset_index(drop=True)
metrodata = metrodata.reset_index().reset_index()[["level_0","CBSA","NAME","CENSUS2010POP"]]
metrodata.rename({"level_0":"rank","CBSA":"cbsacode","NAME":"name","CENSUS2010POP":"population"},axis=1,inplace=True)
metrodata["short_name"] = metrodata["name"].map(lambda s: s.split("-")[0].split(",")[0])

# filter mterodata to top50
top50 = metrodata[metrodata['population'] > metrodata.iloc[50]['population']]

In [3]:
# get fips
metro_to_county = pd.read_csv('../data/cbsa2fipsxw.csv')
metro_to_county = metro_to_county.loc[:,['cbsacode', 'fipsstatecode', 'fipscountycode']].dropna()

metro_to_county['cbsacode'] = metro_to_county['cbsacode'].map(int)
metro_to_county.set_index('cbsacode',inplace=True)

In [4]:
# dataframe with all the information for the queries
to_query = pd.merge(top50, metro_to_county,how='left',left_on='cbsacode',right_index=True)[['rank','short_name','cbsacode','fipsstatecode','fipscountycode']].dropna()
to_query.shape

(426, 5)

In [5]:
# ACS codes for data we are interested in
code_list = {
    "B01003_001":"population",
    "B15003_022":"education_bachelor", 
    "B15003_001":"education_total", 
    "B19301_001":"income",
    "B02001_001":"race_total",
    "B02001_002":"white",
    "B02001_003":"black",
    "B02001_004":"native",
    "B02001_005":"asian"
}

# retrieving the estimates and their errors
mode_list = {
    "E":"",
    "M":"_error"
}

In [6]:
def create_query(code,mode,state,county):
    """
    ACS 5-year estimate API queries.
    
    See https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf for details.
    """
    query = \
        'https://api.census.gov/data/2013/acs/acs5?key=247942759169865ffc196d2f9fce9e9c94fc738b&get='+\
        code+\
        mode+\
        '&&for=tract:*&in=state:'+\
        str(state).zfill(2)+\
        '+county:'+\
        str(county).zfill(3)
        
    return query

In [7]:
def get_data(query):
    """
    For a given ACS code (variable) and mode (estimate/margin of error), using the query, 
    returns a nicely formatted dataframe with the results.
    """
    try:
        # this line returns a response object
        raw_response = request.urlopen(
            query
        )
        # reading the string content from the response object (as if it were an open file)
        str_response = raw_response.read()
        #print(str_response)
    except:
        # jump if there is an error, e.g. because the API blocks us
        str_response = ''

    if str_response!='':
        # if we got a response, parse the string into a list of lists
        parsed_response = json.loads(str_response)
        # first line is the header, rest is the data, convert data to DataFrame
        data = pd.DataFrame(parsed_response[1:])
        # renaming variable name to human readable form
        # parsed_response[0][0] = code_list[code]+mode_list[mode]
        # passing column names to the data
        data.columns =  parsed_response[0]
        # indexing data by geography for join purposes
        data.set_index(['state','county','tract'],inplace=True)
        return data

In [8]:
# creating queries for the selected top metro areas
def create_query_list(state,county):
    """
    Creates all the queries for a given county based on the codes and modes listed in code_list and mode_list.
    """
    return [create_query(code,mode,state,county) for code,mode in product(code_list,mode_list)]

In [10]:
# actually creating all the queries in the dataframe
to_query['queries'] = to_query[['fipsstatecode','fipscountycode']].apply(
    lambda r: create_query_list(int(r['fipsstatecode']),int(r['fipscountycode'])),axis=1
)

In [11]:
header_flag=True

def save_query_results(f,qlist):
    """
    Saving query results into file continuously.
    """
    # use outside reference of header_flag within function
    global header_flag
    # open output file for writing in append mode
    file = open(f,'a')
    try:
        # get parsed response for all queries in query list created earlier
        temp = list(map(get_data,qlist))
    except:
        # if there is no response, print queries
        print("\n".join(qlist))
    try:
        # combine query results into one dataframe, putting education, race etc. columns together
        # rows are different census tracts
        df = pd.concat(temp,axis=1)
        # rename columns from codes to human readable form
        df.columns = [code_list[c[:-1]]+mode_list[c[-1]] for c in df.columns]
        # write combined dataframe to output file
        # if this is the first output, write header
        df.to_csv(file,index=True,header=header_flag)
        # if this is not the first output, omit header
        header_flag=False
        # close file
        file.close()
        return df
    
    except ValueError:
        print("\n".join(qlist))

In [12]:
to_query.shape

(426, 6)

In [16]:
# looping over the desired queries
start_time = time.time()

f = '../data/censusdata-top50-2013.csv'

# clearing file
open(f,'w').close()
header_flag=True
to_query['results'] = to_query['queries'].map(lambda qlist: save_query_results(f,qlist))

print("--- %s seconds ---" % round((time.time() - start_time), 3))

--- 13842.479 seconds ---


In [2]:
# print('\n'.join(to_query['queries'].map(lambda l: '\n'.join(l))))

In [28]:
# read IN all censusdata
base_census2012 = pd.read_csv("../data/censusdata_top50_2012.csv")
census2013 = pd.read_csv('../data/censusdata-top50-2013.csv')
census2014 = pd.read_csv('../data/censusdata-top50-2014.csv')
census2015 = pd.read_csv('../data/censusdata-top50-2015.csv')
census2016 = pd.read_csv('../data/censusdata-top50-2016.csv')
base_census2017 = pd.read_csv("../data/censusdata_top50_2017.csv")

In [29]:
# combine key cols for Gergo
key_cols = ['state', 'county', 'tract', 'population', 'income', 'education_bachelor', 'education_total', 'race_total', 'white', 'black', 'native', 'asian']
base_census2012 = base_census2012[key_cols]
census2013 = census2013[key_cols]
census2014 = census2014[key_cols]
census2015 = census2015[key_cols]
census2016 = census2016[key_cols]
base_census2017 = base_census2017[key_cols]

In [53]:
# merge part
census_combined = pd.merge(base_census2012, census2013, on=["state", "county", "tract"], how="left", suffixes=["2012", "2013"])
census_combined = pd.merge(census_combined, census2014, on=["state", "county", "tract"], how="left")
census_combined = pd.merge(census_combined, census2015, on=["state", "county", "tract"], how="left", suffixes=["2014", "2015"])
census_combined = pd.merge(census_combined, census2016, on=["state", "county", "tract"], how="left")
census_combined = pd.merge(census_combined, base_census2017, on=["state", "county", "tract"], how="left", suffixes=["2016", "2017"])
census_combined.shape

(38788, 57)

In [54]:
# add censustract ID
def create_geoid(row):
    state = str(int(row["state"])).zfill(2)
    county = str(int(row["county"])).zfill(3)
    tract = str(int(row["tract"])).zfill(6)
    return "14000US" +state+county+tract

census_combined['tract_id'] = census_combined.apply(create_geoid, axis=1)

# clean up
cols = census_combined.columns.to_list()
cols = cols[-1:] + cols[:-1]
census_combined = census_combined[cols]
census_combined = census_combined.drop(columns=["state", "county", "tract"])

In [57]:
# drop -666666666.0 values
census_combined = census_combined.replace([-666666666], 0)

In [58]:
# export
census_combined.to_csv("../data/censustract_acs_2012_2017.csv", index=False, sep=";")